In [ ]:
#!/usr/bin/env python
# coding: utf-8
import csv
import json
from bs4 import BeautifulSoup
from bs4 import NavigableString

In [63]:
def process_left(data):
    bs = BeautifulSoup(data, 'html.parser')
    res = {}
    label = None
    
    for c in bs.contents:
        if isinstance(c, NavigableString):
            pass
        elif c.select('img'):
            res['image'] = base + c.a.img.attrs['src']
        elif c.name == 'h6':
            label = c.text[:-1]
        elif label:
            res[label] = c.text.split(';')
            label = None
        elif len(c.text.strip()) > 0:
            t = c.text.strip()
            if label == None and t[-1] == ':':
                label = t[-1]
            elif label:
                res[label] = t
            elif t.find(':') != -1:
                r = t.split(':')
                res[t[0]] = t[1]
            else:
                print('====>', c.text)
    
    return res

In [51]:
def process_bib(data):
    bs = BeautifulSoup(data, 'html.parser')
    res = {
        'bib': data,
        'bib_text': bs.text
    }
    return res

In [52]:
def process_extra(data):
    bs = BeautifulSoup(data, 'html.parser')
    res = {}
    label = None
    
    for c in bs.select("li"):
        key = c.attrs['class'][0]
        link = c.a.attrs['href']
        if link[0] == '/':
            link = base + link

        res[key] = link
    
    return res

In [68]:
url_csv = 'turing_award_page1.csv'

base = 'https://amturing.acm.org'

data = {}

with open(url_csv, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        name = row['name']
        data[name] = {
            'citation': row['citation'],
            'url': row['web-scraper-start-url'],
            'dl_link': row['dl_link-href']
        }

        d = (process_left(row['left']), process_bib(row['bib']), process_extra(row['extra']))
        
        for e in d:
            for k in e:
                data[name][k] = e[k]

#print(json.dumps(data))
#print(data.keys())
print(len(data.keys()))
#print(data['Alan J Perlis'])

with open('./turing_award.json', 'w') as jsonFile:
    json.dump(data, jsonFile, indent=4)


====> The University of Oslo and the University of Aarhus have both named a building after Kristen Nygaard.
70
